In [1]:
%load_ext autoreload
%autoreload 2

In [27]:
# %matplotlib inline

# i had to install this beauty https://github.com/matplotlib/ipympl
# to make following to work ...
%matplotlib widget
import ipywidgets as widgets


# import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import seaborn as sns

import multiprocess as mp

import numpy as np
import pandas as pd
from scipy.linalg import toeplitz
# import cooltools
import cooler
import os
# import re
import time
from functools import partial
from cooltools.lib.numutils import LazyToeplitz

# supress divide by zero warnings and log(0) ...
np.seterr(divide='ignore',invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

### here are some custom functions that we are using downstream - just for clarity ...

In [20]:
def get_chunks(interval,chsize):
    """
    split interval (input as tuple) into
    several intervals of size chsize,
    accounting for boundary condition
    """
    start,end = interval
    int_size = end - start
    for i in range(start, end, chsize):
        if i + chsize < end:
            yield (i,i+chsize)
        else:
            yield (i,end)


def get_extrapolated_array(vals, factor, skip_last=None):
    """
    values - factor - how many times to do it...
    basically turning an array [1,2,3,4] into:
    [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4] when factor=4
    """
    _vals = np.asarray(vals)
    res = np.zeros(factor*len(_vals))
    for shift in range(factor):
        res[shift::factor] = _vals
    # sometimes one'd need to skip several last elements:
    if skip_last is None:
        return res
    else:
        return res[:-skip_last]

def get_proper_interpolated_compartments(clr, ycmp, chrom):
    """
    this is just a wrapper around 'get_extrapolated_array'
    just expand, like 100kb into 1kb array ...
    """
    bin_interval = clr.extent(chrom)
    indices = range(*bin_interval)
    if len(indices) < factor*len(ycmp):
        skipping = (factor*len(ycmp) - len(indices))
        return get_extrapolated_array(ycmp, factor, skipping)
    else:
        return get_extrapolated_array(ycmp, factor)
    
    
def fillcolor_compartment_style(ycmp, ax, bin_range=None, lw=0.5, level=.0, color_less="blue", color_more="red"):
    """
    fill colors between - compartment style ...
    like, yo like, listen, like, you know, like uhmmmmm,
    color like A-like into like red-like, and B-like in
    bluish like you know
    you got the jigitist you know like right yo ?!
    """
    if bin_range is None:
        y = ycmp
        y_size = len(y)
        x = np.arange(y_size)
        y0 = np.zeros_like(x) + level
    else:
        y = ycmp[slice(*bin_range)]
        y_size = len(y)
        x = np.arange(*bin_range)
        y0 = np.zeros_like(x) + level
        
    ax.plot(x,y,color="black",linewidth=lw, alpha=0.5,linestyle="solid",markersize=0)
    ax.fill_between(x,y0,y,where=(y<=level),color = color_less)
    ax.fill_between(x,y0,y,where=(y>level),color = color_more)
    
    return 0


def get_corr_slice(bin_range, region, clr, chr_expect, comps):
    """
    given an interval of bin indices st_slice - first computes
    O/E for each of these bins and correlates it with compartment
    signal.
    """
    O = clr.matrix()[ slice(*bin_range), slice(*clr.extent(region)) ]
    if isinstance(chr_expect, LazyToeplitz):
        local_chrom = region.split(":")[0]
        chr_offset = clr.offset(local_chrom)
        chr_bin_range = [i-chr_offset for i in bin_range]
        chr_region = [i-chr_offset for i in clr.extent(region)]
        E = chr_expect[slice(*chr_bin_range),slice(*chr_region)]
    else:
        # E = get_exp_slice_toeplitz(clr, expect, st_slice, chromosome)
        raise NotImplemented("not implemented yet ...")
    # we get a bunch of obs/exp ...
    oes = pd.DataFrame((O/E).T,columns=range(*bin_range))
    return oes.corrwith(pd.Series(comps))


def get_corr_slice_flank(bin_range, chrom, clr, chr_expect, comps, mask):
    """
    given an interval of bin indices st_slice - first computes
    O/E for each of these bins and correlates it with compartment
    signal.
    """
    O = clr.matrix()[ slice(*bin_range), slice(*clr.extent(chrom)) ]
    chr_offset = clr.offset(chrom)
    chr_bin_range = [i-chr_offset for i in bin_range]
    chr_extent = [i-chr_offset for i in clr.extent(chrom)]
    # check mask and expected - in the lazytoeplitz form:
    if isinstance(chr_expect, LazyToeplitz) and isinstance(mask,LazyToeplitz):
        E = chr_expect[slice(*chr_bin_range),slice(*chr_extent)]
        # mask for expected ...
        M = mask[slice(*chr_bin_range),slice(*chr_extent)]
        # apply mask:
        E[~M] = np.nan
    elif mask is None:
        E = chr_expect[slice(*chr_bin_range),slice(*chr_extent)]
    else:
        # E = get_exp_slice_toeplitz(clr, expect, st_slice, chromosome)
        raise NotImplemented("not implemented yet ...")
    # we get a bunch of obs/exp ...
    oes = pd.DataFrame((O/E).T,columns=range(*bin_range))
    return oes.corrwith(pd.Series(comps))


In [9]:
# this is how we are going to run expected tomorrow ...
# cooltools compute-expected -p 8 -o exp.out -t cis --balance --ignore-diags 2 input.cool

### pick some data to deal with - i like clone 16 2%FA, like, yo!

fabulous, yo

we're like picking yo a compartment track at a given resolution like
and we're also like yo picking a matrix/cooler at the same resolution or higher ...

if it's higher yo - we just like extrapolate compartment track using `factor` yo !

In [10]:
binsize = 100_000
res_human = f"{int(binsize/1000)}kb"

comp_path = "MNP/analysis/compartment"
comp_name = f"poolMNP-bettercis.{res_human}.eigs.cis.vecs.txt"


factor = 1
binsize = 100_000
res_human = f"{int(binsize/1000)}kb"

# comp_path = "MNP/analysis/compartment"
# comp_name = f"poolMNP-bettercis.{res_human}.eigs.cis.vecs.txt"

cool_path = "MNP/data/20200303_MNP_triplicates/results/coolers_library"
cool_name = f"MNP-DT40-1-3-16-2p-R1-T1__galGal5.galGal5.mapq_30.1000.mcool::/resolutions/{binsize}"

exp_path = f"MNP/analysis/expected/{res_human}"
exp_name = f"clone16-2p.{res_human}.expected.cis.tsv"


clr = cooler.Cooler(os.path.join(cool_path,cool_name))
exp = pd.read_csv(os.path.join(exp_path,exp_name),sep='\t')
cmp = pd.read_csv(os.path.join(comp_path,comp_name),sep='\t')

# MNP-DT40-WT1-R1-T1__galGal5.galGal5.mapq_30.1000.mcool
# MNP-DT40-WT2-R1-T1__galGal5.galGal5.mapq_30.1000.mcool

In [11]:
# get chromosomal compartments and interpolated version of those ...
chrom = "chr1"
bin_interval = clr.extent(chrom)
cmp_vals = cmp[cmp["chrom"]==chrom]['E1'].values
interpolated_cmp = get_proper_interpolated_compartments(clr, cmp_vals, chrom)

# lazy expected here - just in case ...
lazy_exp = LazyToeplitz(exp[exp['chrom']==chrom]["balanced.avg"].values)
# #
# flank_bins = 350
# flank_mask = np.zeros(len(interpolated_cmp),dtype=np.bool)
# flank_mask[:flank_bins+1] = True
# mask = LazyToeplitz(flank_mask)


### that's it yo!

some parameters set are irrelevant for later, as we did use ipywidgets yo
which is cool like yo

so you can change parameters on the fly...

In [15]:
# visualize compartments:
fig0 = plt.figure(figsize=(9,4),constrained_layout=True)

# nice gridspec source:
# http://www.sc.eso.org/~bdias/pycoffee/codes/20160407/gridspec_demo.html
spec0 = gridspec.GridSpec(ncols=2, nrows=2, height_ratios=[2,.7], width_ratios=[1,1],figure=fig0)
# # Also make sure the margins and spacing are apropriate
spec0.update(left=0.05, right=0.95, bottom=0.08, top=0.93, wspace=0.02, hspace=0.03)
# # BUT: this is irrelevant for the saved image, if using bbox_inches='tight'in savefig !
ax10 = fig0.add_subplot(spec0[0,0])
ax20 = fig0.add_subplot(spec0[1,:])
ax30 = fig0.add_subplot(spec0[0,1])
# show some compartments and reconstructed checkerboard ...
reconstructed_heatmap = np.outer(cmp_vals,cmp_vals)

# get some O/E ...
oo = clr.matrix().fetch(chrom)
ee = toeplitz(exp[exp["chrom"]==chrom]["balanced.avg"].values)
oe_mat = np.log(oo/ee)

@widgets.interact(vmin=(-1,0.1),vmax=(1,3,.1),level=(-1,1,.1))
def update(vmin=-0.8,vmax=2,level=0):
    ax10.clear()
    ax20.clear()
    ax30.clear()
    # refresh plotting
    ax3m = ax30.imshow(reconstructed_heatmap,cmap="YlOrBr",vmin=vmin,vmax=vmax,aspect='equal')
    ax1m = ax10.imshow(oe_mat,cmap="YlOrBr",vmin=vmin,vmax=vmax)
    fillcolor_compartment_style(cmp_vals, ax20, level=level);


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=-0.8, description='vmin', max=0.1, min=-1.0), FloatSlider(value=2.0, d…

### let's look at some individual loci ...

individual loci to come here right after I'me done with some useless typing

alright!

individual locus interacting with the entire chromosome in cis or just with a large region from it ...


In [26]:
######
# THIS IS USED TO BE WHERE WE SET A BUNCH OF PARAMETERS FOR PLOTTING
# NOW IT'S ALL DONE USING IPYWIDGETS 
######


# chr_offset = clr.offset(chrom)
# chr_extent = [i-chr_offset for i in clr.extent(chrom)]

# # small locus region:
# start, end = 54_900_000, 55_000_000
# locus = f"{chrom}:{start}-{end}"
# locus_extent = [i-chr_offset for i in clr.extent(locus)]

# # larger region:
# flank = flank_bins * binsize
# print(flank)
# start = start - flank if start - flank > 0 else 0
# end = end + flank if end + flank < clr.chromsizes[chrom] else clr.chromsizes[chrom]
# # start, end = 55_900_000, 89_500_000
# region = f"{chrom}:{start}-{end}"
# region_extent = [i-chr_offset for i in clr.extent(region)]

# # locus relative to the region
# region_offset = clr.offset(region)
# locus_region_extent = [i - region_offset for i in clr.extent(locus)]

# # genomic corrdinates - i.e. x:
# x = np.arange(*region_extent)

# print(f"locus {locus} interacting with region {region}")


In [17]:

fig = plt.figure(figsize=(8,4),constrained_layout=True)
spec = gridspec.GridSpec(ncols=1, nrows=5, height_ratios=[1,1,1,1,2],figure=fig)
# # Also make sure the margins and spacing are apropriate
spec.update(left=0.05, right=0.95, bottom=0.08, top=0.93, wspace=0.02, hspace=0.03)
# # BUT: this is irrelevant for the saved image, if using bbox_inches='tight'in savefig !
ax1 = fig.add_subplot(spec[0])
ax2 = fig.add_subplot(spec[1])
ax3 = fig.add_subplot(spec[2])
ax4 = fig.add_subplot(spec[3])
ax5 = fig.add_subplot(spec[4])

@widgets.interact(flank_bins=(10,1000,10),end_add_bins = (1,100,1),chrom=list(clr.chromnames),start=(0,clr.chromsizes[chrom],binsize))
def update(flank_bins=10,end_add_bins=1,start=54_900_000,chrom="chr1"):
    #
    chr_offset = clr.offset(chrom)
    chr_extent = [i-chr_offset for i in clr.extent(chrom)]

    # small locus region:
    start = start
    end = start+end_add_bins*binsize
    locus = f"{chrom}:{start}-{end}"
    locus_extent = [i-chr_offset for i in clr.extent(locus)]

    # larger region:
    flank = flank_bins * binsize
    #     print(flank)
    start = start - flank if start - flank > 0 else 0
    end = end + flank if end + flank < clr.chromsizes[chrom] else clr.chromsizes[chrom]
    # start, end = 55_900_000, 89_500_000
    region = f"{chrom}:{start}-{end}"
    region_extent = [i-chr_offset for i in clr.extent(region)]

    # locus relative to the region
    region_offset = clr.offset(region)
    locus_region_extent = [i - region_offset for i in clr.extent(locus)]

    # genomic corrdinates - i.e. x:
    x = np.arange(*region_extent)
    
    d = clr.matrix().fetch(locus,region)
    eee = lazy_exp[slice(*locus_extent),slice(*region_extent)]

    #
    ax1.clear()
    ax2.clear()
    ax3.clear()
    ax4.clear()
    ax5.clear()
    #
    vmin,vmax = -9,-5
    # extent: left, right, bottom, top
    extent = [*region_extent,*locus_extent[::-1]]
    ax1.imshow(np.log(d),cmap="YlOrBr",vmin=vmin,vmax = vmax,extent = extent, origin="upper")
    ax2.imshow(np.log(eee),cmap="YlOrBr",vmin=vmin,vmax = vmax,extent = extent)
    ax3.imshow(np.log(d/eee),cmap="coolwarm",vmin=-2,vmax = 2,extent = extent)
    ax4.imshow(d/eee,cmap="coolwarm",vmin=0,vmax = 2,extent = extent)
    #
    fillcolor_compartment_style(interpolated_cmp, ax5, region_extent);
    ax5.axvspan(*locus_extent,alpha=0.3,label=region)
    ax5.legend(loc="best")
    ax5.set_xlim(region_extent)
    fig.suptitle(f"{locus} interacting with {region}, log(o), log(e), log(o/e), o/e, EV1, bins {binsize} bp")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=10, description='flank_bins', max=1000, min=10, step=10), IntSlider(valu…

In [18]:
fig2 = plt.figure(figsize=(10,4),constrained_layout=True)
spec2 = gridspec.GridSpec(ncols=3, nrows=3, height_ratios=[1,1,1],width_ratios=[2,1,0.05],figure=fig2)
# # Also make sure the margins and spacing are apropriate
spec2.update(left=0.05, right=0.95, bottom=0.08, top=0.93, wspace=0.02, hspace=0.03)
# # BUT: this is irrelevant for the saved image, if using bbox_inches='tight'in savefig !
ax12 = fig2.add_subplot(spec2[0,0])
ax22 = fig2.add_subplot(spec2[1,0])
ax32 = fig2.add_subplot(spec2[2,0])
ax42 = fig2.add_subplot(spec2[:,1])
ax2_cmap = fig2.add_subplot(spec2[:,2])

@widgets.interact(flank_left_bins=(3,1000,10),flank_right_bins=(3,1000,10),chrom=list(clr.chromnames),locus=(0,clr.chromsizes[chrom],binsize),scale=['log','linear'])
def update_oe(flank_left_bins=11,flank_right_bins=11,locus=44_900_000,chrom="chr1",scale='linear'):
    ax12.clear()
    ax22.clear()
    ax32.clear()
    ax42.clear()
    ax2_cmap.clear()

    # depends only on the chromosome
    chr_offset = clr.offset(chrom)
    
    # get a single bp level focus:
    locus_start = clr.offset((chrom,locus,None)) #locus_extent[0] + chr_offset
    # locus name:
    locus_name = f"{chrom}:{locus}"

    # larger region:
    flank_left = flank_left_bins * binsize
    flank_right = flank_right_bins * binsize
    start = locus - flank_left if locus - flank_left > 0 else 0
    end = locus + flank_right if locus + flank_right < clr.chromsizes[chrom] else clr.chromsizes[chrom]
    region = f"{chrom}:{start}-{end}"
    region_extent = [i-chr_offset for i in clr.extent(region)]
    # genomic corrdinates - i.e. x:
    x = np.arange(*region_extent)

    # extracting a 1D observed ...
    d, = clr.matrix()[locus_start, slice(*clr.extent(region))]
    eee, = lazy_exp[locus_start - chr_offset, slice(*region_extent)]
    if scale == "linear":
        oe = d/eee
        oe_label = "o/e"
        base_level = [1,1]
    elif scale == "log":
        oe = np.log(d/eee)
        oe_label = "log(o/e)"
        base_level = [0,0]
    else:
        print("whaaaaat?!")
    
    ax12.plot(x,np.log(d),'ro-',label="log(o)",alpha=0.5,markersize=3)
    ax12.plot(x,np.log(eee),'b-',label="log(e)")
    ax12.set_xlim(region_extent)
    ax12.legend(loc='best',frameon=False)

    ax22.plot(x,oe,'go-',label=oe_label,alpha=0.5,markersize=3)
    ax22.plot(region_extent,base_level,'k-',label="o=e")
    ax22.set_xlim(region_extent)
    ax22.legend(loc='best',frameon=False)

    fillcolor_compartment_style(interpolated_cmp, ax32, region_extent);
    ax32.axvline(locus_start - chr_offset, label=locus)
    ax32.set_xlim(region_extent)
    ax32.legend(loc="best",frameon=False)

    inter_cmp_inrange = interpolated_cmp[slice(*region_extent)]
    some_corrs = pd.DataFrame({"x":inter_cmp_inrange,"y":oe}).corr()
    r1 = some_corrs["x"]["y"]

    sss = np.abs(locus_start - chr_offset - x)*binsize/1_000_000
    scat = ax42.scatter(inter_cmp_inrange,oe,c=sss,alpha=0.7,label=f"r={r1}",cmap="cividis")
    ax42.set_ylabel(oe_label)
    ax42.legend(loc="best")
    fig2.colorbar(scat,cax=ax2_cmap,label=f"genomic distance from {locus_name}, mb")
    # this one below - was causing all y-axes to shrink tremendously:
    # ax2_cmap.set_ylabel(f"genomic distance from locus {locus}, megabases")
    
    fig2.suptitle(f"{locus_name} interacting with {region}: obs, exp, EV1, o/e and EV1 correlations")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=11, description='flank_left_bins', max=1000, min=3, step=10), IntSlider(…

### now we correlate it on a massive scale ...

In [24]:
# chsize must be small enough to fit in the memory ...
chunk_size = 600
nproc = 16
work_chunks = list(get_chunks(bin_interval,chunk_size))
print(f"tackling {len(work_chunks)} chunks of {chunk_size} bins using {nproc} workers ...")


buffer = {}
#############

figC = plt.figure(figsize=(10,4),constrained_layout=True)
specC = gridspec.GridSpec(ncols=2, nrows=3, height_ratios=[1,1,1],width_ratios=[4,1],figure=figC)
# # Also make sure the margins and spacing are apropriate
specC.update(left=0.05, right=0.95, bottom=0.08, top=0.93, wspace=0.02, hspace=0.03)
# # BUT: this is irrelevant for the saved image, if using bbox_inches='tight'in savefig !
ax1C = figC.add_subplot(specC[0,0])
ax2C = figC.add_subplot(specC[1,0])
ax3C = figC.add_subplot(specC[2,0])

# ax4C = figC.add_subplot(specC[0,1])
# ax5C = figC.add_subplot(specC[1,1])
# # ax_cmap = fig.add_subplot(spec[:,2])

chrom_offset = clr.offset(chrom)
region_extent = [ i-chrom_offset for i in clr.extent(chrom) ]

x = np.arange(len(interpolated_cmp))
y0 = np.zeros_like(interpolated_cmp)



@widgets.interact(
    flank_bins=widgets.IntSlider(
        value=300,min=100,max=10000,step=100,continuous_update=False),
    ylim = widgets.FloatRangeSlider(
        value=[-0.7, 0.8],min=-1,max=1.2,step=0.01,continuous_update=False),
    rerun = False,
    lw = (0,1,0.1),
    range_zoom=widgets.IntRangeSlider(
        value=region_extent,min=0,max=region_extent[1],step=100,continuous_update=False)
)
def update(flank_bins, ylim, rerun, range_zoom, lw=0.1):
    ax1C.clear()
    ax2C.clear()
    ax3C.clear()
    
    if rerun:
        # re-define mask on the fly ...
        flank_mask = np.zeros(len(interpolated_cmp),dtype=np.bool)
        flank_mask[:flank_bins+1] = True
        mask = LazyToeplitz(flank_mask)

        p = mp.Pool(processes=nproc)

        t1 = time.time()
        # get corrs flanked ...
        job = partial(
                get_corr_slice_flank,
                chrom=chrom,
                clr=clr,
                chr_expect=lazy_exp,
                comps=interpolated_cmp,
                mask = mask
               )
        corrs = p.map(job, work_chunks)#, chunksize=chunks)
        t2 = time.time()
        # print(f"done corrs in {t2-t1} sec")
        display(f"done corrs in {t2-t1} sec")


        t1 = time.time()
        # get corrs gw ...
        job = partial(
                get_corr_slice,
                region=chrom,
                clr=clr,
                chr_expect=lazy_exp,
                comps=interpolated_cmp,
               )
        corrs2 = p.map(job, work_chunks)#, chunksize=chunks)
        t2 = time.time()
        display(f"done corrs2 in {t2-t1} sec")

        p.close()
        p.terminate()

        corrs = pd.concat(corrs)
        delta=pd.concat(corrs2)
        
        buffer["corrs"] = corrs
        buffer["delta"] = delta

    # ax1
    fillcolor_compartment_style(interpolated_cmp, ax1C, range_zoom, lw);
    #ax1C.legend(loc="best")
    ax1C.set_xlim(range_zoom)
    ax1C.set_ylabel("EV1")

    # ax2
    fillcolor_compartment_style(buffer["corrs"], ax2C, range_zoom, lw);
    #ax2C.legend(loc="best")
    ax2C.set_xlim(range_zoom)
    ax2C.set_ylabel("pearson-R-flank")
    ax2C.set_ylim(*ylim)

    # ax3
    fillcolor_compartment_style(buffer["delta"], ax3C, range_zoom, lw);
    #ax3C.legend(loc="best")
    ax3C.set_xlim(range_zoom)
    ax3C.set_ylabel("pearson-R-gw")
    ax3C.set_ylim(*ylim)

    # ax4.scatter(corrs,interpolated_cmp,alpha=0.2)
    # ax4.axvline(0,color='red')
    # ax4.axhline(0,color='red')
    # ax5.scatter(delta,interpolated_cmp,alpha=0.2)
    # ax5.axvline(0,color='red')
    # ax5.axhline(0,color='red')
    
    


# t1 = time.time()
# #get deltas ...
# job = partial(
#         get_delta_slice,
#         chromosome=region,
#         clr=clr,
#         chr_expect=lazy_exp,
#         comps=inter_cmp_inrange 
#        )
# delta = p.map(job, work_chunks)
# t2 = time.time()
# print(f"done deltas in {t2-t1} sec")


tackling 4 chunks of 600 bins using 16 workers ...


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=300, continuous_update=False, description='flank_bins', max=10000, min=1…

## legacy and records ...

### figure out flanking mask here - just for records

In [ ]:

chrom = "chr20"
local_chrom = chrom
flank_bins = 3

len_ll = len(exp[exp['chrom']==chrom]["balanced.avg"].values)
M = np.zeros(len_ll,dtype=np.bool)
M[:flank_bins+1] = True
llll = LazyToeplitz(M)

# plt.imshow(toeplitz([True,False,False,False],[True,True,True,True,True,True,True,False,False,False,False,False,False,False,False,False]))
print(clr.extent("chr20"))


# bin_range=[8756,8756+10]
bin_range=[8756+20,8756+26]
# bin_range=[8898-26,clr.extent("chr20")[1]]
# bin_range = clr.extent("chr20")
chrom = "chr20"
local_chrom = chrom
chr_expect=lazy_exp




O = clr.matrix()[ slice(*bin_range), slice(*clr.extent(chrom)) ]
chr_offset = clr.offset(local_chrom)
chr_bin_range = [i-chr_offset for i in bin_range]
chr_extent = [i-chr_offset for i in clr.extent(chrom)]
E = chr_expect[slice(*chr_bin_range),slice(*chr_extent)]
# mask for expected ...
mask_rows = (np.arange(*chr_bin_range)<=flank_bins)
start = chr_bin_range[0]-flank_bins if chr_bin_range[0]>flank_bins else 0
end = chr_bin_range[0]+flank_bins+1 if chr_bin_range[0]+flank_bins<chr_extent[1] else None
print(mask_rows)
print(end)
print(start)
mask_cols = np.zeros(chr_extent[1],dtype=np.int)
mask_cols[start:end] = 1


M1 = toeplitz(mask_rows,mask_cols)
M1 = np.array(M1,dtype=np.bool)

# OO = O.copy()

# OO[~M1] = np.nan


M2 = llll[slice(*chr_bin_range),slice(*chr_extent)]

for i in range(*chr_bin_range):
    j = i
    i -= chr_bin_range[0]
    M1[i,j] = False
    M2[i,j] = False
    



# # we get a bunch of obs/exp ...
f,(ax1,ax2,ax3) = plt.subplots(nrows=3)
# oes = pd.DataFrame((O/E).T,columns=range(*bin_range))
f.set_size_inches(48,5)

ax1.imshow(np.log(O/E))
ax2.imshow(M1)
ax3.imshow(M2)

(M1==M2).all()

In [25]:

## some legacy functions from previously used "locus_exp_module", just for records:

## from locus_exp_module import *


# def get_locus_obs(clr, locus_bins_id, region):
#     """
#     given a bin_id of a locus
#     yield a 1D array of interaction frequency
#     with the region ...
#     """
#     # unpack chrom start stop
#     reg_start,reg_end = clr.extent(region)
#     # it should be 1 row  - so we'll get it to avoid extracting matrix:
#     obs_bin, = clr.matrix()[locus_bins_id, reg_start:reg_end]
#     return obs_bin


# def get_locus_exp(clr,clr_exp,locus_bins_id, chrom):
#     """
#     given a bin_id of a locus
#     yield a 1D array of "expected" interaction frequency
#     with the region ...
#     seems correct based on LazyToeplitz, but rather inefficient
#     """
#     # extract only the relevant part of expected:
#     chrom_exp = clr_exp[clr_exp['chrom']==chrom][["diag","balanced.avg"]].set_index("diag")["balanced.avg"]

#     # our clocus exp to be filled ...
#     locus_exp = np.zeros(len(chrom_exp))

#     # our locus id will be aligned with diag=0
#     # upstream of that we need to fill with reversed
#     # distance decay of size: "locus_bin - chrom_start"
#     # and downstream of that we'll fill with the
#     # distance decay of size: "chrom_end - locus_bin":
#     chrom_start,chrom_end = clr.extent(chrom)

#     downstream = locus_bins_id - chrom_start
#     upstream = chrom_end - locus_bins_id
    
#     # filling in:
#     # reversed downstream
#     locus_exp[0:downstream+1] = chrom_exp.loc[0:downstream][::-1].values
#     # upstream:
#     locus_exp[downstream+1:] = chrom_exp.loc[1:upstream-1].values

#     # beware of numpy indexing and pandas .loc indexing differences
#     # numpy.array([1,2,3,4])[:3] -> [1,2,3]
#     # pd.Series([1,2,3,4],index=[0,1,2,3]).loc[:3] = [1,2,3,4]
#     return locus_exp



# def get_local_score(arr,idx,radius=100):
#     """
#     get some local averge score around idx in arr
#     kinda useless
#     """
#     if idx<radius:
#         return np.nanmean(arr[:idx+radius+1])
#     elif idx > len(arr)-radius:
#         return np.nanmean(arr[idx-radius:])
#     else:
#         return np.nanmean(arr[idx-radius:idx+radius+1])

# def get_corr(st, chromosome, aclr, expect, comps):
#     O, = aclr.matrix()[ st, slice(*aclr.extent(chromosome)) ]
#     E = get_locus_exp(aclr, expect, st, chromosome)
#     a_corr = pd.DataFrame({"x":O/E, "y":comps}).dropna().corr(method="pearson")['x']['y']
#     return float(a_corr)


# def get_delta(st, chromosome, aclr, expect, comps):
#     O, = aclr.matrix()[ st, slice(*aclr.extent(chromosome)) ]
#     E = get_locus_exp(aclr, expect, st, chromosome)
#     return float(get_delta_score(O/E,comps))


# def get_delta_score(OE, cmp_status):
#     """get some local averge score around idx in arr"""
#     return np.nanmean(OE[cmp_status>0]) - np.nanmean(OE[cmp_status<0])


# def get_delta_slice(st_slice, chromosome, clr, expect, comps, level=.0):
#     O = clr.matrix()[ slice(*st_slice), slice(*clr.extent(chromosome)) ]
#     E = get_exp_slice_toeplitz(clr, expect, st_slice, chromosome)
#     return np.nanmean((O/E)[:,comps>level],axis=1) - np.nanmean((O/E)[:,comps<=level],axis=1)

# def get_exp_slice(clr, clr_exp, st_slice, chrom):
#     """
#     given a slice of bin_id-s for several loci
#     yield a 2D array of "expected" interaction frequency
#     within the region ...
#     seems correct based on LazyToeplitz, but probably inefficient
#     because of the "for"-loop
#     """
#     # extract only the relevant part of expected:
#     chrom_exp = clr_exp[clr_exp['chrom']==chrom][["diag","balanced.avg"]].set_index("diag")["balanced.avg"]

#     # our clocus exp to be filled ...
#     start,end = st_slice
#     shape = (end-start, len(chrom_exp))
#     locus_exp = np.zeros(shape)

#     # our locus id will be aligned with diag=0
#     # upstream of that we need to fill with reversed
#     # distance decay of size: "locus_bin - chrom_start"
#     # and downstream of that we'll fill with the
#     # distance decay of size: "chrom_end - locus_bin":
#     chrom_start, chrom_end = clr.extent(chrom)
    
#     for locus_bins_id in range(start, end):
#         downstream = locus_bins_id - chrom_start
#         upstream = chrom_end - locus_bins_id
        
#         rel_id = locus_bins_id - start

#         # filling in:
#         # reversed downstream
#         locus_exp[rel_id, 0:downstream+1] = chrom_exp.loc[0:downstream][::-1].values
#         # upstream:
#         locus_exp[rel_id, downstream+1:] = chrom_exp.loc[1:upstream-1].values
#     return locus_exp


# def get_exp_slice_toeplitz(clr, clr_exp, st_slice, chrom):
#     """
#     given a slice of bin_id-s for several loci
#     yield a 2D array of "expected" interaction frequency
#     within the region ...
#     seems correct based on LazyToeplitz, more efficient
#     because toeplitz function usage ...
#     """
#     # extract only the relevant part of expected:
#     chrom_exp = clr_exp[clr_exp['chrom']==chrom][["diag","balanced.avg"]].set_index("diag")["balanced.avg"]

#     # our clocus exp to be filled ...
#     start,end = st_slice
#     shape = (end-start, len(chrom_exp))
#     cols_exp = np.zeros(len(chrom_exp))

#     chrom_start, chrom_end = clr.extent(chrom)
    
#     locus_bins_id = start
#     downstream = locus_bins_id - chrom_start
#     upstream = chrom_end - locus_bins_id
#     # filling in:
#     # reversed downstream
#     cols_exp[0:downstream+1] = chrom_exp.loc[0:downstream][::-1].values
#     # upstream:
#     cols_exp[downstream+1:] = chrom_exp.loc[1:upstream-1].values
    
#     rows_exp = chrom_exp.loc[downstream:downstream+(end-start)-1].values
#     return toeplitz(rows_exp, cols_exp)


# def get_corr_slice_old(st_slice, chromosome, clr, expect, comps):
#     """
#     given an interval of bin indices st_slice - first computes
#     O/E for each of these bins and correlates it with compartment
#     signal.
#     """
#     O = clr.matrix()[ slice(*st_slice), slice(*clr.extent(chromosome)) ]
#     E = get_exp_slice_toeplitz(clr, expect, st_slice, chromosome)
#     # we get a bunch of obs/exp ...
#     oes = pd.DataFrame((O/E).T,columns=range(*st_slice))
#     return oes.corrwith(pd.Series(comps))
